In [6]:
import math
from IPython.display import display, clear_output
import ipywidgets as widgets

class TicTacToe:
    def __init__(self):
        self.board = [" " for _ in range(9)]  # Initialize an empty board
        self.human_player = "X"
        self.ai_player = "O"
        self.buttons = [widgets.Button(description=str(i+1)) for i in range(9)]  # Buttons for interactive play

    def print_board(self):
        # Display the board in Colab using markdown
        board_view = "\n".join(
            "| " + " | ".join(self.board[i:i+3]) + " |" for i in range(0, 9, 3)
        )
        display(widgets.HTML(f"<pre>{board_view}</pre>"))

    def available_moves(self):
        return [i for i, spot in enumerate(self.board) if spot == " "]

    def make_move(self, position, player):
        self.board[position] = player

    def check_winner(self, player):
        # Check rows, columns, and diagonals for a winner
        for i in range(3):
            # Check rows
            if all(self.board[i*3 + j] == player for j in range(3)):
                return True
            # Check columns
            if all(self.board[i + j*3] == player for j in range(3)):
                return True
        # Check diagonals
        if all(self.board[i] == player for i in [0, 4, 8]) or all(self.board[i] == player for i in [2, 4, 6]):
            return True
        return False

    def check_draw(self):
        return " " not in self.board

    def game_over(self):
        # Checks if any player has won or if it's a draw
        return self.check_winner(self.human_player) or self.check_winner(self.ai_player) or self.check_draw()

    def minimax(self, depth, maximizing_player):
        if self.check_winner(self.human_player):
            return -1
        if self.check_winner(self.ai_player):
            return 1
        if self.check_draw():
            return 0

        if maximizing_player:
            max_eval = -math.inf
            for move in self.available_moves():
                self.make_move(move, self.ai_player)
                eval = self.minimax(depth + 1, False)
                self.make_move(move, " ")
                max_eval = max(max_eval, eval)
            return max_eval
        else:
            min_eval = math.inf
            for move in self.available_moves():
                self.make_move(move, self.human_player)
                eval = self.minimax(depth + 1, True)
                self.make_move(move, " ")
                min_eval = min(min_eval, eval)
            return min_eval

    def get_best_move(self):
        best_move = None
        best_eval = -math.inf
        for move in self.available_moves():
            self.make_move(move, self.ai_player)
            eval = self.minimax(0, False)
            self.make_move(move, " ")
            if eval > best_eval:
                best_eval = eval
                best_move = move
        return best_move

    def play(self):
        print("Welcome to Tic-Tac-Toe! You are 'X', and the AI is 'O'.")

        # Set up interactive buttons
        for i, button in enumerate(self.buttons):
            button.disabled = False  # Enable all buttons
            button.description = str(i+1)  # Reset button labels
            button.style.button_color = None  # Reset button colors
            button.on_click(self.create_click_handler(i))

        self.print_board()
        display(widgets.HBox(self.buttons))

    def create_click_handler(self, index):
        def on_click(btn):
            if self.board[index] != " ":  # Spot already taken
                return

            # Human player's move
            self.make_move(index, self.human_player)
            self.update_buttons()

            if self.game_over():
                self.end_game()
                return

            # AI's move
            ai_move = self.get_best_move()
            self.make_move(ai_move, self.ai_player)
            self.update_buttons()

            if self.game_over():
                self.end_game()

        return on_click

    def update_buttons(self):
        clear_output(wait=True)
        self.print_board()
        for i, button in enumerate(self.buttons):
            button.disabled = self.board[i] != " "  # Disable button if spot is taken
            if self.board[i] == "X":
                button.style.button_color = "lightblue"
                button.description = "X"
            elif self.board[i] == "O":
                button.style.button_color = "lightgreen"
                button.description = "O"
        display(widgets.HBox(self.buttons))

    def end_game(self):
        if self.check_winner(self.human_player):
            print("Congratulations! You win!")
        elif self.check_winner(self.ai_player):
            print("AI wins! Better luck next time.")  # Fixed: AI win message is displayed
        elif self.check_draw():
            print("It's a draw!")  # Draw scenario
        self.update_buttons()
        for button in self.buttons:
            button.disabled = True  # Disable all buttons at the end

if __name__ == "__main__":
    game = TicTacToe()
    game.play()


HTML(value='<pre>| X | O | X |\n| X | O |   |\n|   | O |   |</pre>')